In [1]:
#IMPORT PACKAGES
from rdflib import Graph
import rdflib
import os.path
import importlib
from pathlib import Path
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import multiprocessing
import random as rd
import pandas as pd
# from tabulate import tabulate
import cv2
import laspy
import json
from scipy.spatial.transform import Rotation   
import copy
import geomapi
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl
from scipy.linalg import orthogonal_procrustes
import matplotlib.pyplot as plt
import geomapi.tools.progresstools as pt

from sklearn.cluster import DBSCAN
from PIL import Image


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## INPUTS

In [13]:
name='05_MedOffice_01_F2'

path=Path(os.getcwd()).parents[0]/'data'
pcd_input_path=os.path.join(path, 't0', 'results',f'{name}_pcd.laz')
class_file=path/'_classes.json'

# name=name.split('_')[0]
# json_output_path=os.path.join(path,f'{name}_walls.json') 
# geometry_output_path= os.path.join(path,f'{name}_walls.obj') # these are the bounding surfaces of the reference levels (optional)
output_dir = "C:\Data\temp"

graphPath=str(os.path.join(path, 't0', 'results',f'{name}_graph.ttl'))

grid_resolution = 0.01

Import Classes

In [14]:
# Read the JSON file
with open(class_file, 'r') as file:
    json_data = json.load(file)

# Create a dictionary
class_dict = {
    'classes': json_data['classes'],
    'default': json_data['default'],
    'type': json_data['type'],
    'format': json_data['format'],
    'created_with': json_data['created_with']
}
print(class_dict)

{'classes': [{'name': 'Unassigned', 'id': 255, 'temp_id': 0, 'color': '#9da2ab'}, {'name': 'Floors', 'id': 0, 'temp_id': 1, 'color': '#03c2fc'}, {'name': 'Ceilings', 'id': 1, 'temp_id': 2, 'color': '#e81416'}, {'name': 'Walls', 'id': 2, 'temp_id': 3, 'color': '#ffa500'}, {'name': 'Columns', 'id': 3, 'temp_id': 4, 'color': '#faeb36'}, {'name': 'Doors', 'id': 4, 'temp_id': 5, 'color': '#79c314'}, {'name': 'Windows', 'id': 5, 'temp_id': 6, 'color': '#4b369d'}], 'default': 255, 'type': 'semantic_segmentation', 'format': 'kitti', 'created_with': {'name': 'Saiga', 'version': '1.0.1'}}


Import Graph

In [15]:
graph=Graph().parse(graphPath)
nodes=tl.graph_to_nodes(graph)
wallBIMNodes=[n for n in nodes if 'Walls' in n.subject and type(n)==BIMNode]
wallPCDNodes=[n for n in nodes if 'Walls' in n.subject and type(n)==PointCloudNode]
ceilingsNodes=[n for n in nodes if 'Ceilings' in n.subject and type(n)==PointCloudNode]
floorsNodes=[n for n in nodes if 'Floors' in n.subject and type(n)==PointCloudNode]
clutterNodes=[n for n in nodes if 'Clutter' in n.subject and type(n)==PointCloudNode]
print(f'{len(wallBIMNodes)} wallNodes detected!')
print(f'{len(wallPCDNodes)} wallNodes detected!')
print(f'{len(ceilingsNodes)} ceilingsNodes detected!')
print(f'{len(floorsNodes)} floorsNodes detected!')
print(f'{len(clutterNodes)} clutterNodes detected!')

0 wallNodes detected!
0 wallNodes detected!
0 ceilingsNodes detected!
0 floorsNodes detected!
0 clutterNodes detected!


In [ ]:
print({key:value for key, value in wallBIMNodes[1].__dict__.items() if not key.startswith('__') and not callable(key)})

Import PCD

In [ ]:
laz=laspy.read(pcd_input_path)

match point clouds with graph

In [ ]:
for n in clutterNodes:#+ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    red = laz['red'][idx]
    green = laz['green'][idx]
    blue = laz['blue'][idx]
    #if color is 32 bit, only keep 8 bit color
    if red.max()>255:
        red = laz['red'][idx] >> 8 & 0xFF
        green = laz['green'][idx] >> 8 & 0xFF
        blue = laz['blue'][idx] >> 8 & 0xFF
    # if colorspace is [0-255] -> remap to [0-1]
    if red.max() >1:
        red=red/255
        green=green/255
        blue=blue/255
    pcd.colors=o3d.utility.Vector3dVector(np.vstack((red,green,blue)).transpose())

    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0]

In [ ]:
for n in wallPCDNodes:#+ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    pcd.paint_uniform_color([0.5,0.5,0.5])
    
    red = laz['red'][idx]
    green = laz['green'][idx]
    blue = laz['blue'][idx]
    #if color is 32 bit, only keep 8 bit color
    if red.max()>255:
        red = laz['red'][idx] >> 8 & 0xFF
        green = laz['green'][idx] >> 8 & 0xFF
        blue = laz['blue'][idx] >> 8 & 0xFF
    # if colorspace is [0-255] -> remap to [0-1]
    if red.max() >1:
        red=red/255
        green=green/255
        blue=blue/255
    
    pcd.colors=o3d.utility.Vector3dVector(np.column_stack((red, green, blue)))

    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,0,0] 

match PointCloudNodes to BIMNodes

In [ ]:
for n in wallBIMNodes:
    n.derivedFrom = next(p for p in wallPCDNodes if p.subject.toPython() in [w.derivedFrom for w in wallBIMNodes])

In [ ]:
# joined_pcd=gmu.join_geometries([n.resource.paint_uniform_color(ut.literal_to_array(n.color)) for n in wallPCDNodes if n.resource is not None])
# o3d.visualization.draw_geometries([joined_pcd])

Import Reference Levels

In [ ]:
levelNodes=[n for n in nodes if 'level' in n.subject]
referenceNodes=[]
for l in levelNodes:
    new_graph=ut.get_subject_graph(graph,levelNodes[0].subject)
    n=SessionNode(graph=new_graph)
    n.get_oriented_bounding_box()
    n.resource=o3d.geometry.TriangleMesh.create_from_oriented_bounding_box(n.orientedBoundingBox)
    referenceNodes.append(n) # something is wrong in the tl.graph_to_nodes function
levelNodes=referenceNodes
print(f'{len(levelNodes)} levelNodes detected!')

Import ceilings and floors

In [ ]:
for n in ceilingsNodes+floorsNodes: # this is quite slow because you iterate through 2 scalar fields every time
    idx=np.where((laz['classes']==n.class_id) & (laz['objects']==n.object_id))
    pcd=o3d.geometry.PointCloud()
    pcd.points=o3d.utility.Vector3dVector(laz.xyz[idx])
    n.resource=pcd
    n.get_oriented_bounding_box()
    n.orientedBoundingBox.color=[1,1,0]

## PROCESSING

Convert dtrings from the graph to arrays

In [ ]:
for n in wallBIMNodes:
    n.startpoint = np.asarray(n.startpoint[1:-1].split(), dtype=float)
    n.endpoint = np.asarray(n.endpoint[1:-1].split(), dtype=float)
    n.normal = np.asarray(n.normal[1:-1].split(), dtype=float)
    n.height = float(n.height)
    n.name = n.subject.split('///')[-1]
    

In [ ]:
def points_on_line(point1, point2, step_size):
    """
    Generate points on a line between two given points with a specified step size.

    Parameters:
    - point1: The starting point of the line.
    - point2: The ending point of the line.
    - step_size: The step size between consecutive points.

    Returns:
    - points: A list of points on the line.
    """
    # Calculate the direction vector
    direction = point2 - point1

    # Calculate the length of the line segment
    length = np.linalg.norm(direction)

    # Normalize the direction vector
    direction /= length

    # Calculate the number of steps needed
    num_steps = int(length / step_size)

    # Generate points along the line
    points = np.array([point1 + i * step_size * direction for i in range(num_steps + 1)])

    return points


Load the full resolution point cloud for a more accurate result (unlabeled)

In [ ]:
laz = laspy.read(os.path.join(Path(os.getcwd()).parents[0]/'data',"full_resolution_populierenhof.las"))
full_res_point_cloud_o3d = gmu.las_to_pcd(laz)

Convert the full point cloud into a mesh and add it to a raycasting scene

In [ ]:

#Cut a part out of the full resolution pointcloud
# joined_pcd = full_res_point_cloud_o3d.crop(expanded_bounding_box)
#Create a messh from this point cloud 
octree=pt.pcd_to_octree(full_res_point_cloud_o3d,10) #if octree is None else octree
mesh=gmu.octree_to_voxelmesh(octree) #if mesh is None else mesh


#Create a identity array containing the color so this can be retrieved afterwards
original_colors=np.asarray(mesh.vertex_colors)
indices=np.asarray(mesh.triangles)[:,0]
triangle_colors=original_colors[indices]
#append black color at the end of the array for the invalid hits
triangle_colors=np.vstack((triangle_colors,np.array([0,0,0])))

# Create raycasting scene
scene = o3d.t.geometry.RaycastingScene()
mesh=o3d.t.geometry.TriangleMesh.from_legacy(mesh)
scene.add_triangles(mesh) 

# Calculate the size of each octree node based on octree depth and overall size
def calculate_node_size(octree_depth, octree_size):
    num_voxels_per_dim = 2 ** octree_depth
    voxel_size = octree_size / num_voxels_per_dim
    return voxel_size

# Example usage:
octree_depth = octree.max_depth  # Example value for max_depth
octree_size = octree.size  # Example size of the octree in world units
voxel_size = calculate_node_size(octree_depth, octree_size)


In [ ]:
def is_door(openingwidth, openingheight, t_min_width = 0.5, t_max_width = 3, t_min_height = 1.5, t_max_height = 2.3):

    if t_min_width < openingwidth < t_max_width and t_min_height < openingheight < t_max_height:
        return True
    else: 
        return False

## Method 1: Using the pointcloud and wall data to retrieve potential openings in the walls 
(Can also be used to retrieve wall detailing)

In [ ]:
# def create_opening_pcds(startpoint, endpoint, height, resolution, direction, scene, offset =0.7, path = None, show = False, min_samples = 5000, eps = 0.2, wallThickness = 0.1, voxel_size = 0.01):
    
#     min_z = np.min([endpoint[2], startpoint[2]])
#     max_z = min_z + height
#     num_z_steps = int(height /resolution)
#     z_grid = np.linspace(min_z, max_z, num_z_steps)  # Adjust the number of grid points as needed
#     xyz_grid = []
#     for z in z_grid:
#         start = n.startpoint.copy()
#         end =  n.endpoint.copy()
#         start[2] = z
#         end[2] = z
#         xyz_grid.append(points_on_line(start, end, resolution))

#     grid = np.asarray(xyz_grid).reshape((-1, 3), order='C') 
    
#     openingpcds = []
#     # Create Open3D point cloud
#     grid_center_pcd = o3d.geometry.PointCloud()
#     grid_center_pcd.points = o3d.utility.Vector3dVector(np.asarray(grid))
#     grid_center_pcd.paint_uniform_color([1,0,0])

#     #In face is the dominant side of the wall
#     grid_in = grid + direction*offset
#     #out face is the other side of the dominant side
#     grid_out = grid - direction*offset

#     #create rays for the in side (towards the dominant side
#     ori_x = direction[0] * np.ones(len(grid))
#     ori_y = direction[1] * np.ones(len(grid))
#     ori_z = direction[2] * np.ones(len(grid))
    
#     pos_x = grid_in[:,0]
#     pos_y = grid_in[:,1]
#     pos_z = grid_in[:,2]

#     # Stack the calculated values along the third axis to create the grid
#     rays_in_values = np.stack((pos_x, pos_y, pos_z, -ori_x, -ori_y, -ori_z), axis=1)
#     rays_in_tensor = o3d.core.Tensor(rays_in_values, dtype=o3d.core.Dtype.Float32)

#     pos_x = grid_out[:,0]
#     pos_y = grid_out[:,1]
#     pos_z = grid_out[:,2]

#     rays_out_values = np.stack((pos_x, pos_y, pos_z, ori_x, ori_y, ori_z), axis=1)
#     rays_out_tensor = o3d.core.Tensor(rays_out_values, dtype=o3d.core.Dtype.Float32)       

#     ans_in = scene.cast_rays(rays_in_tensor)
#     ans_out = scene.cast_rays(rays_out_tensor)

#     hits_in = ans_in['t_hit'].numpy()
#     hits_out = ans_out['t_hit'].numpy()

#     opening_points = []
#     opening_colors = []
    
#     colors = np.zeros((len(hits_in), 3))
#     for i, c in enumerate(colors):
#         if hits_in[i] > 2*offset and hits_out[i] > 2*offset:
#             colors[i] = [0,1,0]
#             opening_points.append(grid[i])
#             opening_colors.append([0,1,0])

#         elif hits_in[i] <2*offset or hits_out[i] < 2*offset:
#             colors[i] = [0.5,0.5,0.5]
#             thickness = 2*offset-hits_out[i]-hits_in[i]
    
#             if 1.2*voxel_size < thickness < 0.5*wallThickness:
#                 # colors[i] = [1,0.5,0]
#                 opening_points.append(grid[i])
#                 opening_colors.append([0,1,0])


#     # grid_center_pcd.colors =o3d.utility.Vector3dVector(np.asarray(colors))
#     opening_pcd = o3d.geometry.PointCloud()
#     opening_pcd.points = o3d.utility.Vector3dVector(np.asarray(opening_points))
#     opening_pcd.colors = o3d.utility.Vector3dVector(np.asarray(opening_colors))
    
#     opening_pcd, ind = opening_pcd.remove_radius_outlier(nb_points = 100, radius = 0.1)

#     # o3d.visualization.draw_geometries([opening_pcd])

#     points = np.asarray(opening_pcd.points)
#     if len(points) > min_samples:

#         # Perform clustering using DBSCAN
#         dbscan = DBSCAN(eps=eps, min_samples=min_samples)
#         labels = dbscan.fit_predict(points)

#         # Extract unique cluster labels (excluding noise label -1)
#         unique_labels = np.unique(labels[labels != -1])

#         # Iterate over each cluster label and save corresponding points to a separate point cloud
#         for label in unique_labels:
#             cluster_points = points[labels == label]
#             cluster_pcd = o3d.geometry.PointCloud()
#             cluster_pcd.points = o3d.utility.Vector3dVector(cluster_points)
#             openingpcds.append(cluster_pcd)
#     if show:
#         o3d.visualization.draw_geometries(openingpcds)
#     # o3d.visualization.draw_geometries([grid_center_pcd])
   
#     return openingpcds
    

In [ ]:
# pcd_resolution = 0.01
# for n in wallBIMNodes:
    
#     length = np.sqrt(np.sum((n.endpoint - n.startpoint)**2))
#     surface = length * n.height
#     n.openings1 = []

#     if not surface < 3 and n.height > 1.5 and length > 0.8:
                
#         n.openings1 = create_opening_pcds(startpoint = n.startpoint, endpoint = n.endpoint, height=n.height, resolution= 0.01, direction= n.normal, scene= scene, offset =0.5, path = None, show = False, min_samples = 1000, eps = 0.25, wallThickness = n.wallThickness, voxel_size=voxel_size)
        

In [ ]:
# doors = []
# other = []
# for n in wallBIMNodes:
#     n.baseConstraint = next(l for l in levelNodes if l.subject.toPython() in [w.base_constraint for w in wallBIMNodes])
    
#     if len(n.openings1) > 0:
#         for opening in n.openings1:
#             # Extract the z-coordinates
#             points2 = np.asarray(opening.points)
#             z_values = points2[:, 2]
#             unique_z_values = np.unique(z_values)

#             #Compute the width of the opening
#             opening_width = 0.0
#             for z_value in unique_z_values:
#                 # Get points with the current z-value
#                 points_with_same_z = points2[z_values == z_value]

#                 # Find the outermost points based on XY coordinates
#                 min_x = np.min(points_with_same_z[:, 0])
#                 max_x = np.max(points_with_same_z[:, 0])
#                 min_y = np.min(points_with_same_z[:, 1])
#                 max_y = np.max(points_with_same_z[:, 1])

#                 # Compute the diagonal length of the bounding box
#                 diagonal_length = np.linalg.norm([max_x - min_x, max_y - min_y])

#                 if diagonal_length > opening_width:
#                     opening_width = diagonal_length
                    
#             print("Opening Width:", opening_width)
            
#             #Compute the Height of the door
#             lowest_z = np.min(z_values)
#             highest_z = np.max(z_values)
#             opening_height = highest_z - lowest_z   
#             print("Opening height:", opening_height)
            
#             #Depth should be equal to the wall thickness
            
#             if is_door(openingwidth = opening_width, openingheight = opening_height):
#                 doors.append(opening)
#                 print("This is most likely a door")

In [ ]:
# # o3d.visualization.draw_geometries(doors)
# joined_pcd=gmu.join_geometries(doors)
# o3d.io.write_point_cloud(os.path.join(Path(os.getcwd()).parents[0]/'data', "doors.pcd"), joined_pcd)

## Method 2: Using the pointcloud and wall data to create an ortho foto of the wall and use object detection
(Can also be used to retrieve other elements)

In [ ]:
def create_wall_ortho(startpoint, endpoint, height, resolution, direction, scene, offset =1, path = None, show = False, dominant = True, max_distance = 0.5, min_distance = 0.5):
    image_size = (int(np.sqrt(np.sum((endpoint - startpoint)**2)) / resolution)+1, int(height / resolution))
    
    min_z = np.min([endpoint[2], startpoint[2]])
    max_z = min_z + height
    num_z_steps = int(height /resolution)
    z_grid = np.linspace(min_z, max_z, num_z_steps)  # Adjust the number of grid points as needed
    z_grid = z_grid[::-1]
    xyz_grid = []
    for z in z_grid:
        start = startpoint.copy()
        end =  endpoint.copy()
        start[2] = z
        end[2] = z
        if not dominant:
            xyz_grid.append(points_on_line(start, end, resolution)[::-1])
        else: 
            xyz_grid.append(points_on_line(start, end, resolution))

    grid = np.asarray(xyz_grid).reshape((-1, 3), order='C') 
    ray_grid = grid + direction*offset
    
    #create rays for the in side (towards the dominant side
    ori_x = direction[0] * np.ones(len(ray_grid))
    ori_y = direction[1] * np.ones(len(ray_grid))
    ori_z = direction[2] * np.ones(len(ray_grid))
    
    pos_x = ray_grid[:,0]
    pos_y = ray_grid[:,1]
    pos_z = ray_grid[:,2]
    
    # Stack the calculated values along the third axis to create the grid
    rays_values = np.stack((pos_x, pos_y, pos_z, -ori_x, -ori_y, -ori_z), axis=1)
    rays_tensor = o3d.core.Tensor(rays_values, dtype=o3d.core.Dtype.Float32)
    
    ans= scene.cast_rays(rays_tensor) 
    
    
    triangle_ids = ans["primitive_ids"].numpy() # triangles     
    triangle_ids = triangle_ids.flatten()
    np.put(triangle_ids,np.where(triangle_ids==scene.INVALID_ID),triangle_colors.shape[0]-1) # replace invalid id's by last (which is the above added black color)
    
    # Get the hit distances for each ray
    hit_distances = ans["t_hit"].numpy().flatten()
    
    # Filter out hits that are too far or too close
    if max_distance is not None:
        triangle_ids[hit_distances > max_distance+offset] = triangle_colors.shape[0] - 1  # Set to black
        
    if min_distance is not None:
        triangle_ids[hit_distances < min_distance] = triangle_colors.shape[0] - 1  # Set to black
    
    colors = triangle_colors[triangle_ids]
    ortho = np.reshape(colors,(image_size[1],image_size[0],3))
    
    if show:
        plt.imshow(ortho)
        plt.show()
    if path:
        image = Image.fromarray((ortho * 255).astype(np.uint8))
        # Save the image
        image.save(path)
        
    return ortho

In [ ]:
image_resolution = 0.01

for n in wallBIMNodes:
    
    length = np.sqrt(np.sum((n.endpoint - n.startpoint)**2))
    surface = length * n.height
    image_size = (int(length / image_resolution), int(n.height / image_resolution))
    n.orthos = []
    

    if not surface < 3 and n.height > 1.5 and length > 0.8:  
        #Create an ortho of the dominant side of the wall      
        ortho = create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = n.normal, scene=scene)
        n.orthos.append(ortho)
        #Also create an ortho of the other side of the wall
        if not n.wallThickness == 0.1: #Single faced wall only needs one side
            ortho = create_wall_ortho(startpoint = n.startpoint, endpoint= n.endpoint, height= n.height, resolution = image_resolution, direction = -n.normal, scene=scene, dominant = False)
            n.orthos.append(ortho)

In [ ]:
import torch
from PIL import Image
from torchvision.ops import box_convert

# Grounding DINO
from groundingdino.models import build_model
from groundingdino.util.slconfig import SLConfig
from groundingdino.util.utils import clean_state_dict
from groundingdino.util.inference import annotate, load_image, predict

import supervision as sv

# segment anything
# from segment_anything import build_sam, SamPredictor 
import numpy as np


# diffusers
import torch

from huggingface_hub import hf_hub_download

In [ ]:
def load_model_hf(repo_id, filename, ckpt_config_filename, device='cpu'):
    cache_config_file = hf_hub_download(repo_id=repo_id, filename=ckpt_config_filename)

    args = SLConfig.fromfile(cache_config_file) 
    model = build_model(args)
    args.device = device

    cache_file = hf_hub_download(repo_id=repo_id, filename=filename)
    checkpoint = torch.load(cache_file, map_location='cpu')
    log = model.load_state_dict(clean_state_dict(checkpoint['model']), strict=False)
    print("Model loaded from {} \n => {}".format(cache_file, log))
    _ = model.eval()
    return model   

In [ ]:
# Use this command for evaluate the Grounding DINO model
# Or you can download the model by yourself
ckpt_repo_id = "ShilongLiu/GroundingDINO"
ckpt_filenmae = "groundingdino_swinb_cogcoor.pth"
ckpt_config_filename = "GroundingDINO_SwinB.cfg.py"

In [ ]:
groundingdino_model = load_model_hf(ckpt_repo_id, ckpt_filenmae, ckpt_config_filename, device="cuda")

In [ ]:
count = 0
pointcloud = []


for n in wallBIMNodes:
    TEXT_PROMPT = "Door"
    BOX_TRESHOLD = 0.20
    TEXT_TRESHOLD = 0.5
    
    n.boxes = []
    n.logits = []
    n.phrases = []
    
    if len(n.orthos) > 0:
        
        for ortho in n.orthos:
            boxes = None
            image = load_image(Image.fromarray((ortho * 255).astype(np.uint8)))

            boxes, logits, phrases = predict(
                model=groundingdino_model, 
                image=image, 
                caption=TEXT_PROMPT, 
                box_threshold=BOX_TRESHOLD, 
                text_threshold=TEXT_TRESHOLD
            )
            
            n.boxes.append(boxes)
            n.logits.append(logits)
            n.phrases.append(phrases)                  
                    

In [ ]:
def create_point_cloud(img, startpoint, endpoint, box):
    # Define the UV coordinate ranges
    u_range = np.linspace(0, 1, img.shape[1])  # Width of the image corresponds to U
    v_range = np.linspace(0, 1, img.shape[0])  # Height of the image corresponds to V
    
    if not img[0].shape == img[1].shape:
        print("PROBLEM")
    # Initialize arrays to store points and colors
    points = []
    colors = []

    # Precalculate box boundaries
    box_boundaries_0 = []
    opening_width = int(np.asarray(box)[2]*img.shape[1])* image_resolution
    opening_height =   int(np.asarray(box)[3]*img.shape[0]) * image_resolution
    
    detection_center_u = int(np.asarray(box)[0]*img.shape[1]) * image_resolution
    detection_center_v = int(np.asarray(box)[1]*img.shape[0]) * image_resolution  
                
    u_min = detection_center_u - opening_width / 2
    u_max = detection_center_u + opening_width / 2
    v_min = detection_center_v - opening_height / 2
    v_max = detection_center_v + opening_height / 2
    reference_level = (detection_center_v + opening_height/2)
    
    if is_door(openingwidth = opening_width, openingheight = opening_height) and reference_level > (img.shape[0]*image_resolution-0.5):
        box_boundaries_0.append((u_min, u_max, v_min, v_max))
            

    # Iterate over each pixel in the image
    if len(box_boundaries_0) > 0:
        for v, u, a in np.ndindex(img.shape):
            # Map UV coordinates to XYZ coordinates
            x = (1 - u_range[u]) * startpoint[0] + u_range[u] * endpoint[0]
            y = (1 - u_range[u]) * startpoint[1] + u_range[u] * endpoint[1]
            z = (1 - v_range[v]) * endpoint[2]  # Since V represents Z
            
            # Check if the point falls inside any of the boxes
            in_box_0 = False
            for box_boundary in box_boundaries_0:
                    u_min, u_max, v_min, v_max = box_boundary
                    if u_min <= u*image_resolution < u_max and v_min <= v*image_resolution < v_max:
                        in_box_0 = True
                        break
        
            # Assign the color based on whether the point is inside any of the boxes
            if in_box_0: 
                colors.append([0, 0, 1])  # Red
                points.append([x, y, z])

    return np.array(points), np.array(colors)

In [ ]:
def extract_box_with_margin(image, box, margin = 10):
    # Extract center and size from the detection array
    width = int(np.asarray(box)[2]*image.shape[1])
    height =   int(np.asarray(box)[3]*image.shape[0])
    
    center_x = int(np.asarray(box)[0]*image.shape[1])
    center_y = int(np.asarray(box)[1]*image.shape[0])  
    

    # Calculate coordinates of the bounding box with extra margin
    x1 = int(center_x - width/2) - margin
    y1 = int(center_y - height/2) - margin
    x2 = int(center_x + width/2) + margin
    y2 = int(center_y + height/2) + margin

    # Ensure the box is within the image bounds
    x1 = max(x1, 0)
    y1 = max(y1, 0)
    x2 = min(x2, image.shape[1])
    y2 = min(y2, image.shape[0])

    # Create a new image containing pixels within the bounding box
    extracted_image = np.copy(image[y1:y2, x1:x2])
    normalized_image = cv2.normalize(extracted_image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    return normalized_image.astype(np.uint8)

def compute_center(point1, point2):
    center = [(point1[0] + point2[0]) / 2,
              (point1[1] + point2[1]) / 2,
              (point1[2] + point2[2]) / 2]
    return center

In [ ]:
potential_doors = []
pointcloud = []

wall = 0

for n in wallBIMNodes: #^22:24 interessante muren
    
    count = 0
    stand = 0
    if len(n.orthos) > 0 and (len(n.boxes[0]) > 0 or len(n.boxes[1]) > 0):
        for boxes in n.boxes:
            if len(boxes) > 0: 
                for i, box in enumerate(boxes):
                    if stand == 1:
                        points, colors = create_point_cloud(n.orthos[stand], n.endpoint,n.startpoint, box)
                    else:
                        points, colors = create_point_cloud(n.orthos[stand],n.startpoint, n.endpoint, box)
                        
                    opening_width = int(np.asarray(box)[2]*n.orthos[stand].shape[1])* image_resolution
                            
                    # print("Opening Width:", opening_width)
                    
                    #Compute the Height of the door
                    opening_height =   int(np.asarray(box)[3]*n.orthos[stand].shape[0]) * image_resolution
                    # print("Opening height:", opening_height)
                    
                    detection_center_u = int(np.asarray(box)[0]*n.orthos[stand].shape[1]) * image_resolution
                    detection_center_v = int(np.asarray(box)[1]*n.orthos[stand].shape[0]) * image_resolution                
                    
                    reference_level = (detection_center_v + opening_height/2)
                    
                    if is_door(openingwidth = opening_width, openingheight = opening_height) and reference_level > (n.orthos[stand].shape[0]*image_resolution-0.5):
                        image = extract_box_with_margin(n.orthos[stand], box.unsqueeze(0)[0])
                        image = image[...,::-1] # BGR to RGB
                        #Image.fromarray(image).save(os.path.join(Path(os.getcwd()).parents[0]/'data',(str(wall) +"-"+ str(stand)+"-" + str(count) +'-DOOR.png')))
                        #annotated_frame = annotate(image_source= n.orthos[stand], boxes=box.unsqueeze(0), logits=n.logits[stand][i].unsqueeze(0), phrases=[n.phrases[stand][i]])
                        # annotated_frame = annotated_frame[...,::-1] # BGR to RGB
                        # Image.fromarray(annotated_frame).save(os.path.join(Path(os.getcwd()).parents[0]/'data',(str(wall) +"-"+ str(stand)+"-" + str(count) +'-DOOR.png')))
                        print("%s : Door => Referencelevel: %s; Width: %s; Height: %s" %((str(wall) + "-" + str(stand) + "-" + str(count)), reference_level, opening_width, opening_height))
                        count += 1
                        if points.size > 0:
                            # Create Open3D point cloud
                            pcd_o3d = o3d.geometry.PointCloud()
                            pcd_o3d.points = o3d.utility.Vector3dVector(points)
                            pcd_o3d.colors = o3d.utility.Vector3dVector(colors)
                            o3d.io.write_point_cloud(os.path.join(Path(os.getcwd()).parents[0]/'data',(str(wall) + "-" + str(stand) + "-" + str(count) + '-DOOR.pcd')), pcd_o3d)
                            base_points = points[points[:, 2] == 0, :2]
                            # print(base_points)
                            min_x_index = np.argmin(points[:, 0])
                            max_x_index = np.argmax(points[:, 0])
                                                        
                            startpoint = [points[min_x_index][0],points[min_x_index][1],0 ]
                            endpoint = [points[max_x_index][0],points[max_x_index][1],0 ]
                            
                            center = compute_center(startpoint, endpoint)
                            distance = np.sqrt((endpoint[0] - startpoint[0])**2 + (endpoint[1] - startpoint[1])**2)
                            
                            
                            # pcd_node = PointCloudNode(resource = pcd_o3d, path = os.path.join(Path(os.getcwd()).parents[0]/'data',(str(wall) + "-" + str(stand) + "-" + str(count) + '-DOOR.pcd')))
                            
                        on = Node()
                        on.width = opening_width
                        on.height = opening_height
                        on.depth = n.wallThickness
                        on.pcd = pcd_o3d
                        on.image = image
                        on.startpoint = startpoint
                        on.endpoint = endpoint
                        on.center = center
                        on.normal = n.normal
                        on.host = n
                        potential_doors.append(on)
            stand += 1
        wall += 1

In [ ]:
def plot_lines(lines, show_centers=True):
    fig, ax = plt.subplots()

    # Iterate over each line and plot it with a different color
    if len(lines[0]) == 2:
        for i, (start, end) in enumerate(lines):
            color = plt.cm.viridis(i / len(lines))  # Get a color from the Viridis colormap
            ax.plot([start[0], end[0]], [start[1], end[1]], color=color)
            
            if show_centers:
                # Compute the center of the line
                center = compute_center(start, end)
                
                # Plot the center with the same color as the line
                ax.scatter(center[0], center[1], color=color, label=f'Center {i+1}')

        ax.set_aspect('equal', adjustable='box')  # Set aspect ratio to equal
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_title('Lines with Centers')
        if show_centers:
            ax.legend()
        plt.show()
    else:
        for i, (start, end, _) in enumerate(lines):
            color = plt.cm.viridis(i / len(lines))  # Get a color from the Viridis colormap
            ax.plot([start[0], end[0]], [start[1], end[1]], color=color)
            
            if show_centers:
                # Compute the center of the line
                center = compute_center(start, end)
                
                # Plot the center with the same color as the line
                ax.scatter(center[0], center[1], color=color, label=f'Center {i+1}')

        ax.set_aspect('equal', adjustable='box')  # Set aspect ratio to equal
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_title('Lines with Centers')
        if show_centers:
            ax.legend()
        plt.show()

In [ ]:
def compute_normal(start, end):
    """
    Compute the normal vector of a line given its start and end points.
    
    Parameters:
        start (tuple): The start point of the line.
        end (tuple): The end point of the line.
    
    Returns:
        numpy.ndarray: The computed normal vector.
    """
    # Compute the direction vector of the line
    direction = np.array(end) - np.array(start)
    # Compute the normal vector by rotating the direction vector 90 degrees
    normal = np.array([-direction[1], direction[0]])
    # Normalize the normal vector
    normal /= np.linalg.norm(normal)
    
    return normal

# Function to calculate the center point of a line segment
def calculate_center(line):
    startpoint, endpoint, _ = line
    return [(startpoint[0] + endpoint[0]) / 2, (startpoint[1] + endpoint[1]) / 2]

# Function to calculate the distance between two points
def calculate_distance(point1, point2):
    return np.linalg.norm(np.array(point1) - np.array(point2))

In [ ]:
# Function to cluster lines based on normals and centers
def cluster_lines(lines, distance_threshold=1, equal_normals = True):
    
    cluster_ids = [-1] * len(lines)
    current_cluster_id = 0
    if equal_normals:
        normals = set(tuple(line[2]) for line in lines)  # Convert numpy array to tuple
        for normal in normals:
            lines_with_normal = [line for line in lines if np.array_equal(line[2], normal)]
            
            centers = [calculate_center(line) for line in lines_with_normal]
            
            # Using DBSCAN for clustering centers
            dbscan = DBSCAN(eps=distance_threshold, min_samples=1, metric=calculate_distance)
            labels = dbscan.fit_predict(centers)
            
            unique_labels = set(labels)
            for label in unique_labels:
                cluster_lines = [lines_with_normal[i] for i in range(len(lines_with_normal)) if labels[i] == label]
                for line in cluster_lines:
                    line_index = None
                    for i, l in enumerate(lines):
                        if np.array_equal(l, line):
                            line_index = i
                            break
                    cluster_ids[line_index] = current_cluster_id
                current_cluster_id += 1
    else:            
        centers = [calculate_center(line) for line in lines]
        
        # Using DBSCAN for clustering centers
        dbscan = DBSCAN(eps=distance_threshold, min_samples=1, metric=calculate_distance)
        labels = dbscan.fit_predict(centers)
        
        unique_labels = set(labels)
        for label in unique_labels:
            cluster_lines = [lines[i] for i in range(len(lines)) if labels[i] == label]
            for line in cluster_lines:
                line_index = None
                for i, l in enumerate(lines):
                    if np.array_equal(l, line):
                        line_index = i
                        break
                cluster_ids[line_index] = current_cluster_id
            current_cluster_id += 1
    
    return cluster_ids

Eerste clustering om detecties langst beide kanten te bundelen

In [ ]:
def find_outer_points(points):
    # Convert points to numpy array for easier manipulation
    points = np.array(points)
    
    # Find the index of the point with the smallest coordinate along the line
    min_index = np.argmin(points[:, 0])  # Assuming the line is along the x-axis
    
    # Find the index of the point with the largest coordinate along the line
    max_index = np.argmax(points[:, 0])  # Assuming the line is along the x-axis
    
    # Retrieve the two outer points
    outer_point_min = points[min_index]
    outer_point_max = points[max_index]
    
    return outer_point_min, outer_point_max

In [ ]:
def retrieve_all_clusters_indices(cluster_labels):
    all_clusters_indices = []
    
    # Get unique cluster labels
    unique_labels = np.unique(cluster_labels)
    
    # Iterate through each unique cluster label
    for cluster_label in unique_labels:
        if not cluster_label == -1:
            cluster_indices = [i for i, label in enumerate(cluster_labels) if label == cluster_label]
            all_clusters_indices.append(cluster_indices)
        else: 
            for i, label in enumerate(cluster_labels):
                if label == cluster_label:
                    all_clusters_indices.append([i])
    
    return all_clusters_indices

def are_normals_same_direction(objects):
    # Check if all normals have the same direction
    reference_normal = objects[0].normal
    for obj in objects[1:]:
        if not np.allclose(obj.normal, reference_normal):
            return False
    return True

def merge_objects(openings):
    if not are_normals_same_direction(openings):
        on = openings[0]
        return on
    
    # Merge objects by finding extreme start and end points
    on = Node()
    on.startpoint, on.endpoint = find_outer_points([obj.startpoint for obj in openings] + [obj.endpoint for obj in openings])
    on.height = sum(n.height for n in openings)/len(openings)
    on.depth = openings[0].depth
    on.pcd = [n.pcd for n in openings]
    on.image = [n.image for n in openings]
    on.center = compute_center(startpoint, endpoint)
    on.normal = openings[0].normal
    on.host = openings[0].host
    on.width = np.sqrt((on.endpoint[0] - on.startpoint[0])**2 + (on.endpoint[1] - on.startpoint[1])**2)
    
    return on

In [ ]:
lines = []

for n in potential_doors:
    lines.append([n.startpoint, n.endpoint, n.normal])

plot_lines(lines)
clustered_ids = cluster_lines(lines) 
# plot_clustered_lines_separately(lines, clustered_ids)
all_clusters_indices = retrieve_all_clusters_indices(clustered_ids)
clustered_lines = []

for cluster_indices in all_clusters_indices:
    openings = []
    for i in cluster_indices:
        openings.append(potential_doors[i])
    on = merge_objects(openings)
    clustered_lines.append(on)
    
lines = []
for n in clustered_lines:
    lines.append([n.startpoint, n.endpoint])

plot_lines(lines)

In [ ]:
lines = []

for n in clustered_lines:
    lines.append([n.startpoint, n.endpoint, n.normal])
    
clustered_ids = cluster_lines(lines, distance_threshold=0.5, equal_normals = True)
all_clusters_indices = retrieve_all_clusters_indices(clustered_ids)

clustered_lines2 = []
for cluster_indices in all_clusters_indices:
    openings = []
    for i in cluster_indices:
        openings.append(clustered_lines[i])
    on = merge_objects(openings)
    clustered_lines2.append(on)
    
lines = []
for n in clustered_lines2:
    lines.append([n.startpoint, n.endpoint])

plot_lines(lines)

In [ ]:
def calculate_percentage_black_pixels(image):
    # Convert the image to grayscale
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Count black pixels
    black_pixel_count = cv2.countNonZero(grayscale_image)

    # Calculate total number of pixels
    total_pixels = grayscale_image.shape[0] * grayscale_image.shape[1]

    # Calculate percentage of black pixels
    percentage_black_pixels = (black_pixel_count / total_pixels) * 100

    return percentage_black_pixels

In [ ]:
lines = []

for n in clustered_lines2:
    lines.append([n.startpoint, n.endpoint, n.normal])
    
clustered_ids = cluster_lines(lines, distance_threshold=1.2, equal_normals = False)
all_clusters_indices = retrieve_all_clusters_indices(clustered_ids)

openings = []
for cluster in all_clusters_indices:
    if len(cluster) > 1:
        l = [clustered_lines2[i] for i in cluster]
        
        # Plotting the images
        # fig, axes = plt.subplots(1, len(l))            
        
        lines = []
        i = 0
        for j, n in enumerate(l):
            lines.append([n.startpoint, n.endpoint, n.normal])
            # axes[i].imshow(Image.fromarray(n.image[0][0]))
            # axes[i].axis('off')
            i+=1
            if calculate_percentage_black_pixels(n.image[0][0]) < 50:
                openings.append(n)
                for jj, n2 in enumerate(l): 
                    if not jj == j:
                        if np.array_equal(n.normal, n2.normal):
                            openings.append(n2)
                
        # plot_lines(lines)
    else:
        openings.append(clustered_lines[cluster[0]])

In [ ]:
lines =[]
for n in openings:
    lines.append([n.startpoint, n.endpoint, n.normal])
plot_lines(lines, show_centers=False)